# Sentiment Classification & Embedding II

* Embedding Layer
* Sequence Model

# 새로운 전처리

In [ ]:
import nltk
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
def mecab_tokenizer(text):
    pos = mecab_ko.pos(text)
    return [word_pos[0] for word_pos in pos if word_pos[1] not in ('VA', 'VV', 'IC', 'VX', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JC', 'EP', 'EF')]


In [ ]:
# count_vectorizer = CountVectorizer(tokenizer= mecab_tokenizer)
# x_train_counts = count_vectorizer.fit_transform(x_train)ㅁ

# 시작

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

In [ ]:
!pip install mecab-python3

In [ ]:
# MeCab 사전 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from mecab import MeCab
import numpy as np
import pandas as pd

# 데이터셋 로드
data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/train.csv')

In [ ]:
# data['label'] = data['label'].apply(lambda x: '코드' if x.startswith('코드') else x)

In [ ]:
data.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2


In [ ]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

In [ ]:
pp_df = data.replace({'label':label_dict}).copy()

In [ ]:
pp_df.tail(6)

,text,label
3700,"사용자를 만들때, 사용자에게 권한을 직접 주고 나서, 같은 권한을 가지고 있는 그...",1
3701,"토큰화 이후 train val 를 분리하고 각 train set, val set에 ...",0
3702,올린 값들 중 최고점인 건가요? 아니면 최근에 올린 파일로 무조건 갱신인가요?\n최...,3
3703,수업에서 cacoo랑 packet tracer를 배우는 이유가\n\n1. IT 인프...,2
3704,inplace =True 해도 값이 변경이 안되고 none으로 뜹니다. 혹시 원격지...,4
3705,상관관계는 그렇게 크지 않게 나오는데 p -value 값은 관련이 있게 나오는데 \...,2


In [ ]:
# 토큰화
mecab = MeCab()
stop_words = ['은', '는', '이', '가', '을', '를', '금일', '안녕', '일차', '간과', '가요', '감사', '구합', '차', 'eda', 'ㄱ', 'ㄴ', 'ㄷ', 'ㅅ', 'ㅁ', 'ㄷ', 'ㅂ', '후', '시', '거', '의',  '그', '때', '로', '요', '과', '걸', '중', '안', '은', '제', '왜', '결', '수', '컴', '터', '제하', '고생']


In [ ]:
import urllib.request

url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.txt'
urllib.request.urlretrieve(url, './stopwords-ko.txt')

with open('./stopwords-ko.txt', encoding='utf-8') as f:
    stop_words = set([line.strip() for line in f.readlines()])


from konlpy.tag import Komoran

# komoran = Komoran()

main_pos = []
# 형태소 분석
for text in pp_df['text']:
    pos = mecab.pos(text)
    # 동사와 조사를 제외한 단어 추출
    words = [word for word, tag in pos if tag not in ['VV', 'VA', 'VX', 'VCP', 'VCN', 'JX']]
    main_words = ' '.join(words)
    main_pos.append(main_words)
print(main_pos)
stop_words = ['은', '는', '이', '가', '을', '를', '금일', '안녕', '일차', '간과', '가요', '감사', '구합', '차', 'eda', 'ㄱ', 'ㄴ', 'ㄷ', 'ㅅ', 'ㅁ', 'ㄷ', 'ㅂ', '후', '시', '거', '의',  '그', '때', '로', '요', '과', '걸', '중', '안', '은', '제', '왜', '결', '수', '컴', '터', '제하', '고생']

# 불용어 처리
main_pos = [[w for w in s if w not in stop_words] for s in main_pos]

data['main_pos'] = main_pos

['self . convs 1 = nn . ModuleList ( [ nn . Conv 2 d ( 1 , Co , ( K , 100 ) ) for K in Ks ] ) 1 . 여기 서 커널 사이즈 K , 100 은 한 단어 의 최대 길이 가 100 이 지 는다는 가정 으로 그냥 100 으로 선언 한 것 인지 궁금 합니다 .', '현재 이미지 를 여러 개 업로드 기 위해 자바 스크립트 로 동적 으로 폼 여러 개 생성 하 는 데 클릭 해도 생 성 이 안 네요 . .. 막혀서 뭘 할 수 가 네요 . . 혹시나 해서 기본 예제 코드 로 간단히 돌려도 안 됩니다 . 이유 가 뭘까요', 'glob . glob ( PATH ) 를 사용 할 때 질문 입니다 . PATH 에 [ ] 가 포함 되 면 제대로 작동 하 지 는 것 아 질문 드립니다 . 제공 해 신 파일 에 AIVLE _ 3 rd [ Team ] 폴더 가 포함 되 어 는데 , 이 폴더 안 에 glob 이 정상 적 으로 작동 하 지 습니다 .', "tmpp = tmp . groupby ( by = ' Addr 1 ', as _ index = False ) [ [ ' Category ']]. sum ( ) plt . figure ( figsize =( 15 , 5 ) ) plt . bar ( tmpp [ ' Addr 1 '], tmpp [ ' Category ']/ 1000 ) plt . ticklabel _ format ( axis =' y ', style =' plain ') plt . show ( ) 오류 : unsupported operand type ( s ) for / : ' str ' and ' int ' 게 해결 하 면 을까요 ? !!!!", 'filename = TEST _ IMAGE + str ( round ( frame _ sec ) ) + ". jpg " 이렇게 면 test _ path 안 에 image 폴더 안 에 이미지 들 이 저장 되 는 걸로 이해 했 는데 이렇게 실행 하 니 test 폴더 에 

In [ ]:
data.head(5)

,text,label,main_pos
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드,self . convs 1 = nn . ModuleList ( [ nn . Conv...
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹,현재 이미지 를 여러 개 업로드 기 위해 자바 스크립트 로 동적 으로 폼 여러 개 ...
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드,glob . glob ( PATH ) 를 사용 할 때 질문 입니다 . PATH 에 ...
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드,"tmpp = tmp . groupby ( by = ' Addr 1 ', as _ i..."
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드,filename = TEST _ IMAGE + str ( round ( frame ...


In [ ]:

# 데이터셋 전처리
# data = data.dropna()
# data = data.drop_duplicates()
# data = data.reset_index(drop=True)

# 토큰화
mecab = MeCab()
stop_words = ['의', '에게','인가가', '한테','조차차' ,'에도','은', '는', '이', '가', '을', '를', '금일', '안녕', '일차', '간과', '가요', '감사', '구합', '차', 'eda', 'ㄱ', 'ㄴ', 'ㄷ', 'ㅅ', 'ㅁ', 'ㄷ', 'ㅂ', '후', '시', '거', '의',  '그', '때', '로', '요', '과', '걸', '중', '안', '은', '제', '왜', '결', '수', '컴', '터', '제하', '고생']

tokenizer = Tokenizer(num_words=28000)
tokenizer.fit_on_texts(pp_df['text'].apply(lambda x: ' '.join(mecab.morphs(x))))
sequences = tokenizer.texts_to_sequences(pp_df['text'].apply(lambda x: ' '.join(mecab.morphs(x))))

# 불용어 처리
sequences = [[w for w in s if w not in stop_words] for s in sequences]




In [ ]:
# 주격 조사: -이/가
# 보격 조사: -을/를
# 관형격 조사: -의
# 목적격 조사: -에게/-한테, -을/를
# 부사격 조사: -에/-에서/-에도/-조차
# 호격 조사: -아/-야/-이여/-여
# 서술격 조사: -이다/-이다/-인가
# 능격 조사: -을/를/-아/-어/-여/-고

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# nltk.download('stopwords')

# stop_words = set(stopwords.words('korean'))

# tokenized_text = word_tokenize(data['text'])

# filtered_text = [word for word in tokenized_text if not word in stop_words]

In [ ]:
max_len = max([len(item) for item in sequences])
print(max_len)

957


In [ ]:
pp_df['label']

0       0
1       1
2       0
3       0
4       0
       ..
3701    0
3702    3
3703    2
3704    4
3705    2
Name: label, Length: 3706, dtype: int64

In [ ]:
# 패딩
max_length = 957
#max_length = 6874

# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre', truncating='post')
padded_sequences_pos = pad_sequences(sequences, maxlen=max_length, padding='post')
padded_sequences_pre = pad_sequences(sequences, maxlen=max_length, padding='pre')
# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
# padded_sequences2 = pad_sequences(main_pos, maxlen=max_length, padding='pre')


# 벡터화
X_pos = padded_sequences_pos
X_pre = padded_sequences_pre

# # 라벨 인코딩
# encoder = LabelEncoder()
# encoder.fit(pp_df['label'])
# y = encoder.transform(pp_df['label'])

In [ ]:
# 원핫인코딩딩
from tensorflow.keras.utils import to_categorical
y = to_categorical(pp_df['label'], 5)

In [ ]:
y

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
pp_df['label']

0       0
1       1
2       0
3       0
4       0
       ..
3701    0
3702    3
3703    2
3704    4
3705    2
Name: label, Length: 3706, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_pre, y, test_size=0.2, random_state=123)

In [ ]:
x_train.shape, y_train.shape

((2964, 957), (2964, 5))

In [ ]:
# x_train.reset_index(drop=True, inplace=True)
# y_train.reset_index(drop=True, inplace=True)
# x_test.reset_index(drop=True, inplace=True)
# y_test.reset_index(drop=True, inplace=True)

In [ ]:
# 문장 최대 길이 구하기
text_lengths = [len(data) for text in data]
pp_df['text'].apply(len).max()

6012

In [ ]:
max_len = max([len(item) for item in X])
print(max_len)

957


In [ ]:
x_train.max(), x_test.max()

(9736, 9754)

In [ ]:
max_words = 20000 
embedding_dim = 128 ## 단어 embedding 차원
max_len = 957 ## 문장 최대 길이

# test_df

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/test.csv')

In [ ]:
test_df.head()

,id,text
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...
1,1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향..."
2,2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...
3,3,"def max4(a,b,c,d):\n maximum = a\n if b&..."
4,4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      929 non-null    int64 
 1   text    929 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB


In [ ]:

# 데이터셋 전처리
# data = data.dropna()
# data = data.drop_duplicates()
# data = data.reset_index(drop=True)

# 토큰화
mecab = MeCab()
stop_words = ['의', '에게','인가가', '한테','조차차' ,'에도','은', '는', '이', '가', '을', '를', '금일', '안녕', '일차', '간과', '가요', '감사', '구합', '차', 'eda', 'ㄱ', 'ㄴ', 'ㄷ', 'ㅅ', 'ㅁ', 'ㄷ', 'ㅂ', '후', '시', '거', '의',  '그', '때', '로', '요', '과', '걸', '중', '안', '은', '제', '왜', '결', '수', '컴', '터', '제하', '고생']

# tokenizer_test = Tokenizer(num_words=28000)
# tokenizer_test.fit_on_texts(test_df['text'].apply(lambda x: ' '.join(mecab.morphs(x))))
sequences = tokenizer_test.texts_to_sequences(test_df['text'].apply(lambda x: ' '.join(mecab.morphs(x))))
# tokenizer를 fit하면 안 되고 모델 학습시킬 때 사용한 tokenizer를 그대로 사용해야한다!
# 불용어 처리
sequences_test = [[w for w in s if w not in stop_words] for s in sequences_test]




In [ ]:
# 패딩
max_length = 957
#max_length = 6874

# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre', truncating='post')
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length, padding='pre')
# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
# padded_sequences2 = pad_sequences(main_pos, maxlen=max_length, padding='pre')


# 벡터화
X_test = padded_sequences_test

# # 라벨 인코딩
# encoder = LabelEncoder()
# encoder.fit(test_df['label'])
# y = encoder.transform(test_df['label'])

929

## 모델링 시작

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool1D
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, SimpleRNN, GRU

from tensorflow.keras.models import Sequential

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# # 모델 정의
# keras.backend.clear_session()
# model = Sequential()
# model.add(Embedding( max_words, # 사용할 총 token개수
#                     embedding_dim, # 한 token당 몇차원(몇개)의 특징값으로 바꾸어낼 것인가.
#                     input_length=max_len )) # 통일한 문장 길이
# # model.add(Embedding(10000, 128, input_length=max_length))
# model.add(Conv1D(128, 5, activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(len(encoder.classes_), activation='softmax'))

# # 모델 컴파일
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# 모델 정의
keras.backend.clear_session()
model = Sequential()
model.add(Embedding( max_words, # 사용할 총 token개수
                    embedding_dim, # 한 token당 몇차원(몇개)의 특징값으로 바꾸어낼 것인가.
                    input_length=max_len )) # 통일한 문장 길이
# model.add(Embedding(10000, 128, input_length=max_length))
model.add(Conv1D(64, 5, activation='relu'))
model.add(keras.layers.LSTM(128, activation='tanh'))
model.add(keras.layers.Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, restore_best_weights=True)

In [ ]:
model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose =1, callbacks=[es])

Epoch 1/100
70/70 [==============================] - 32s 267ms/step - loss: 1.3206 - accuracy: 0.4345 - val_loss: 1.1119 - val_accuracy: 0.5126
Epoch 2/100
70/70 [==============================] - 13s 185ms/step - loss: 0.9508 - accuracy: 0.5983 - val_loss: 0.9840 - val_accuracy: 0.5701
Epoch 3/100
70/70 [==============================] - 14s 195ms/step - loss: 0.6725 - accuracy: 0.7472 - val_loss: 0.9231 - val_accuracy: 0.6655
Epoch 4/100
70/70 [==============================] - 10s 142ms/step - loss: 0.3953 - accuracy: 0.8619 - val_loss: 0.9058 - val_accuracy: 0.6960
Epoch 5/100
70/70 [==============================] - 6s 90ms/step - loss: 0.1971 - accuracy: 0.9343 - val_loss: 1.1261 - val_accuracy: 0.7050
Epoch 6/100
70/70 [==============================] - 7s 101ms/step - loss: 0.1087 - accuracy: 0.9699 - val_loss: 1.2153 - val_accuracy: 0.7338
Epoch 7/100
70/70 [==============================] - 7s 95ms/step - loss: 0.0616 - accuracy: 0.9798 - val_loss: 1.1919 - val_accuracy: 0.74

In [ ]:
from sklearn.metrics import f1_score
y_pred = model.predict(x_test)

29/29 [==============================] - 1s 18ms/step


In [ ]:
test_pr = y_pred
test_te = y_test

In [ ]:
test_pr = np.argmax(y_pred, axis=1)
# test_te = np.argmax(y_test, axis=1)

In [ ]:
f1_mean = f1_score(test_te, test_pr, average=None).mean()
print(f1_mean)

0.6921636023748282


## 모델2

In [ ]:
keras.backend.clear_session()

model2 = Sequential()

model2.add(Embedding( max_words, # 사용할 총 token개수
                    embedding_dim, # 한 token당 몇차원(몇개)의 특징값으로 바꾸어낼 것인가.
                    input_length=max_len )) # 통일한 문장 길이

model2.add(Conv1D(64, 5, activation='swish')) 
forward_layer = LSTM(32, return_sequences=True )
backward_layer = GRU(32, return_sequences=True, go_backwards=True)
model2.add(Bidirectional(forward_layer, backward_layer=backward_layer))
# model2.add(Bidirectional(GRU(128, return_sequences=True)))
model2.add(Flatten())
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='swish'))
model2.add(Dense(32, activation='swish'))
# model2.add(Dense(128, activation='swish'))
model2.add(Dense(len(encoder.classes_), activation='softmax'))

# 모델 컴파일
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose =1, callbacks=[es])

Epoch 1/100
70/70 [==============================] - 35s 427ms/step - loss: 1.7973 - accuracy: 0.2627 - val_loss: 1.5069 - val_accuracy: 0.4191
Epoch 2/100
70/70 [==============================] - 31s 443ms/step - loss: 1.2332 - accuracy: 0.5164 - val_loss: 1.0445 - val_accuracy: 0.5845
Epoch 3/100
70/70 [==============================] - 27s 386ms/step - loss: 0.7847 - accuracy: 0.7063 - val_loss: 0.9619 - val_accuracy: 0.6619
Epoch 4/100
70/70 [==============================] - 26s 369ms/step - loss: 0.4397 - accuracy: 0.8489 - val_loss: 0.9510 - val_accuracy: 0.7194
Epoch 5/100
70/70 [==============================] - 25s 360ms/step - loss: 0.2368 - accuracy: 0.9276 - val_loss: 1.1774 - val_accuracy: 0.6978
Epoch 6/100
70/70 [==============================] - 25s 354ms/step - loss: 0.1396 - accuracy: 0.9582 - val_loss: 1.2593 - val_accuracy: 0.6960
Epoch 7/100
70/70 [==============================] - 24s 338ms/step - loss: 0.0799 - accuracy: 0.9730 - val_loss: 1.5495 - val_accuracy:

In [ ]:
y_pred2 = model2.predict(x_test)
test_pr2 = y_pred2
test_te2 = y_test
test_pr2 = np.argmax(y_pred2, axis=1)
f1_mean = f1_score(test_te2, test_pr2, average=None).mean()
print(f1_mean)

## 모델3

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# # 모델 정의
# keras.backend.clear_session()
# model3 = Sequential()
# model3.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
# model3.add(Conv1D(filters=64, kernel_size=5, activation='swish'))
# model3.add(GlobalMaxPooling1D())
# model3.add(Dropout(0.2))  # Dropout layer 추가
# model3.add(Dense(units=64, activation='swish'))
# model3.add(Dropout(0.2))  # Dropout layer 추가
# model3.add(Dense(units=len(encoder.classes_), activation='softmax'))

# # 모델 컴파일
# model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model3.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose =1, callbacks=[es])

Epoch 1/100
70/70 [==============================] - 14s 181ms/step - loss: 1.6633 - accuracy: 0.2955 - val_loss: 1.4828 - val_accuracy: 0.3381
Epoch 2/100
70/70 [==============================] - 11s 152ms/step - loss: 1.3522 - accuracy: 0.4831 - val_loss: 1.1790 - val_accuracy: 0.5989
Epoch 3/100
70/70 [==============================] - 7s 102ms/step - loss: 0.8589 - accuracy: 0.7188 - val_loss: 0.7683 - val_accuracy: 0.7500
Epoch 4/100
70/70 [==============================] - 8s 115ms/step - loss: 0.4151 - accuracy: 0.8772 - val_loss: 0.6653 - val_accuracy: 0.7896
Epoch 5/100
70/70 [==============================] - 6s 87ms/step - loss: 0.1877 - accuracy: 0.9492 - val_loss: 0.6902 - val_accuracy: 0.7716
Epoch 6/100
70/70 [==============================] - 6s 83ms/step - loss: 0.0981 - accuracy: 0.9748 - val_loss: 0.7456 - val_accuracy: 0.7824
Epoch 7/100
70/70 [==============================] - 6s 88ms/step - loss: 0.0771 - accuracy: 0.9829 - val_loss: 0.8066 - val_accuracy: 0.7896


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# 모델 정의
keras.backend.clear_session()
model3 = Sequential()
model3.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model3.add(Conv1D(filters=64, kernel_size=5, activation='swish'))
model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.2))  # Dropout layer 추가
model3.add(keras.layers.Flatten())
model3.add(Dense(32, activation='swish'))
model3.add(Dropout(0.2))
model3.add(Dense(units=len(encoder.classes_), activation='softmax'))

# 모델 컴파일
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.summary()
model3.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose =1, callbacks=[es])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 957, 128)          3200000   
                                                                 
 conv1d (Conv1D)             (None, 953, 64)           41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                        

In [ ]:
y_pred3 = model3.predict(x_test)
test_pr3 = y_pred3
test_te3 = y_test
test_pr3 = np.argmax(y_pred3, axis=1)
f1_mean = f1_score(test_te3, test_pr3, average=None).mean()
print(f1_mean)

29/29 [==============================] - 0s 3ms/step
0.8368888656636244


In [ ]:
x_train.shape

(2964, 957)

# 모델4

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add

# # 모델 정의
# inputs = Input(shape=(max_len,))
# embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)
# conv1 = Conv1D(filters=256, kernel_size=5, activation='relu', padding='same')(embedding)
# dropout1 = Dropout(0.2)(conv1)
# conv2 = Conv1D(filters=256, kernel_size=5, activation='relu', padding='same')(dropout1)
# residual = Add()([conv1, conv2])  # Residual connection
# pooling = GlobalMaxPooling1D()(residual)
# dropout2 = Dropout(0.2)(pooling)
# dense1 = Dense(units=64, activation='relu')(dropout2)
# dropout3 = Dropout(0.2)(dense1)
# outputs = Dense(units=len(encoder.classes_), activation='softmax')(dropout3)

# model4 = Model(inputs=inputs, outputs=outputs)

# # 모델 컴파일
# model4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add

# # 모델 정의
# inputs = Input(shape=(max_len,))
# embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)
# conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(embedding)
# dropout1 = Dropout(0.3)(conv1)
# # conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(dropout1)
# # dropout1 = Dropout(0.3)(conv2)
# # residual = Add()([conv1, conv2]) # Residual connection
# pooling = GlobalMaxPooling1D()(dropout1)
# hl = keras.layers.BatchNormalization()(pooling)
# hl = Dropout(0.25)(pooling)
# hl = keras.layers.Flatten()(hl)
# hl = Dense(32, activation='relu')(hl)

# hl = Dropout(0.15)(hl)

# outputs = Dense(units=len(encoder.classes_), activation='softmax')(hl)

# model4 = Model(inputs, outputs)

# # 모델 컴파일
# model4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add

# 모델 정의
inputs = Input(shape=(max_len,))
embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)
conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(embedding)
dropout1 = Dropout(0.2)(conv1)
# conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(dropout1)
# residual = Add()([conv1, conv2])  # Residual connection
pooling = GlobalMaxPooling1D()(dropout1)
hl = Dense(32, activation='relu')(hl)
hl = Dropout(0.2)(pooling)
hl = keras.layers.Flatten()(hl)
hl = Dense(32, activation='relu')(hl)

outputs = Dense(5, activation='softmax')(hl)

model4 = Model(inputs, outputs)

# 모델 컴파일
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 새 섹션

In [ ]:
model4.fit(x_train, y_train, epochs=10000, validation_split=0.2, verbose =1, callbacks=[es], batch_size=32)

Epoch 1/10000
75/75 [==============================] - 12s 144ms/step - loss: 1.3575 - accuracy: 0.4100 - val_loss: 1.2240 - val_accuracy: 0.4199
Epoch 2/10000
75/75 [==============================] - 9s 120ms/step - loss: 0.9892 - accuracy: 0.6078 - val_loss: 0.8857 - val_accuracy: 0.6796
Epoch 3/10000
75/75 [==============================] - 6s 75ms/step - loss: 0.5577 - accuracy: 0.8250 - val_loss: 0.6065 - val_accuracy: 0.7808
Epoch 4/10000
75/75 [==============================] - 8s 100ms/step - loss: 0.2585 - accuracy: 0.9287 - val_loss: 0.5234 - val_accuracy: 0.8044
Epoch 5/10000
75/75 [==============================] - 6s 81ms/step - loss: 0.1197 - accuracy: 0.9734 - val_loss: 0.5130 - val_accuracy: 0.8246
Epoch 6/10000
75/75 [==============================] - 5s 64ms/step - loss: 0.0646 - accuracy: 0.9861 - val_loss: 0.5196 - val_accuracy: 0.8196
Epoch 7/10000
75/75 [==============================] - 3s 41ms/step - loss: 0.0370 - accuracy: 0.9916 - val_loss: 0.5621 - val_accur

In [ ]:
from sklearn.metrics import *
y_pred4 = model4.predict(x_test)
test_pr4 = np.argmax(y_pred4, axis=1)
test_te4 = np.argmax(y_test, axis=1)
f1_mean = f1_score(test_te4, test_pr4, average=None).mean()
print(f1_mean)
#0.8420327738533226

24/24 [==============================] - 0s 3ms/step
0.8611650866870308


In [ ]:
y_pred4.shape

(742, 5)

In [ ]:
y_test.shape

(742, 5)

In [ ]:
answer = model4.predict(X_test)



30/30 [==============================] - 0s 5ms/step


In [ ]:
answer

array([[0.19572416, 0.01097582, 0.7828897 , 0.00735979, 0.00305044],
       [0.03756122, 0.22741504, 0.2992576 , 0.42127585, 0.01449029],
       [0.74129564, 0.21067668, 0.02822771, 0.01449408, 0.00530583],
       ...,
       [0.30381694, 0.41918367, 0.19822739, 0.06566725, 0.01310468],
       [0.11767041, 0.18803792, 0.6226857 , 0.04764172, 0.02396433],
       [0.83275443, 0.04011706, 0.06090527, 0.06338792, 0.00283536]],
      dtype=float32)

In [ ]:
answer_A = np.argmax(answer, axis=1)

In [ ]:
test_df['new'] = answer_A

In [ ]:
f1_score(test_te4, test_pr4, average=None)

array([0.80149813, 0.96774194, 0.76719577, 0.73381295, 0.83561644])

In [ ]:
0.8496902271707942

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드


In [ ]:
# model4.save('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/f1_8640.h5')

## 모델5 -> 모델 4 튜닝

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# # 모델 정의
# keras.backend.clear_session()
# model5 = Sequential()
# model5.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

# model5.add(Conv1D(filters=128, kernel_size=5, activation='swish'))
# # model5.add(GlobalMaxPooling1D())
# model5.add(Dropout(0.2))  # Dropout layer 추가
# model5.add(Dense(units=64, activation='swish'))
# # model5.add(Dense(units=64, activation='swish'))
# model5.add(Dropout(0.2))  # Dropout layer 추가
# model5.add(Dense(units=len(encoder.classes_), activation='softmax'))

# # 모델 컴파일
# model5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# SGD
# model5.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose =1, callbacks=[es])

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, momentum=0.9)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 957, 128)          3200000   
                                                                 
 conv1d (Conv1D)             (None, 953, 64)           41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# 모델 정의
keras.backend.clear_session()
model5 = Sequential()
model5.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

model5.add(Conv1D(filters=64, kernel_size=5, activation='swish'))
model5.add(GlobalMaxPooling1D())
model5.add(Dropout(0.2))  # Dropout layer 추가
model5.add(keras.layers.Flatten())
model5.add(Dense(units=64, activation='swish'))
# model5.add(Dense(units=64, activation='swish'))
# model5.add(Dropout(0.2))  # Dropout layer 추가
model5.add(Dense(units=len(encoder.classes_), activation='softmax'))

# 모델 컴파일
model5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model5.fit(x_train, y_train, epochs=100, validation_split=0.1, verbose =1, callbacks=[es])

Epoch 1/100
79/79 [==============================] - 14s 157ms/step - loss: 1.3986 - accuracy: 0.4286 - val_loss: 1.1576 - val_accuracy: 0.4532
Epoch 2/100
79/79 [==============================] - 10s 122ms/step - loss: 1.0773 - accuracy: 0.5490 - val_loss: 0.9327 - val_accuracy: 0.6403
Epoch 3/100
79/79 [==============================] - 7s 95ms/step - loss: 0.6764 - accuracy: 0.7389 - val_loss: 0.6894 - val_accuracy: 0.7590
Epoch 4/100
79/79 [==============================] - 8s 96ms/step - loss: 0.3281 - accuracy: 0.8912 - val_loss: 0.6601 - val_accuracy: 0.7662
Epoch 5/100
79/79 [==============================] - 5s 68ms/step - loss: 0.1210 - accuracy: 0.9696 - val_loss: 0.7007 - val_accuracy: 0.7914
Epoch 6/100
79/79 [==============================] - 5s 58ms/step - loss: 0.0433 - accuracy: 0.9912 - val_loss: 0.7684 - val_accuracy: 0.8129
Epoch 7/100
79/79 [==============================] - 4s 47ms/step - loss: 0.0178 - accuracy: 0.9984 - val_loss: 0.8890 - val_accuracy: 0.8094
Ep

In [ ]:
y_pred5 = model5.predict(x_test)
test_pr5 = y_pred5
test_te5 = y_test
test_pr5 = np.argmax(y_pred5, axis=1)
f1_mean = f1_score(test_te5, test_pr5, average=None).mean()
print(f1_mean)

NameError: ignored

## 모델6

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# 모델 정의
keras.backend.clear_session()
model6 = Sequential()
model6.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

model6.add(Conv1D(filters=128, kernel_size=5, activation='swish'))
model6.add(GlobalMaxPooling1D())
model6.add(Dropout(0.2))  # Dropout layer 추가
model6.add(Dense(units=64, activation='swish'))
# model5.add(Dense(units=64, activation='swish'))
model6.add(Dropout(0.2))  # Dropout layer 추가
model6.add(Dense(units=len(encoder.classes_), activation='softmax'))

# 모델 컴파일
model6.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model6.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose =1, callbacks=[es])

Epoch 1/100
70/70 [==============================] - 46s 639ms/step - loss: 1.3630 - accuracy: 0.4314 - val_loss: 1.1692 - val_accuracy: 0.4496
Epoch 2/100
70/70 [==============================] - 10s 139ms/step - loss: 1.0710 - accuracy: 0.5677 - val_loss: 0.9450 - val_accuracy: 0.6583
Epoch 3/100
70/70 [==============================] - 8s 116ms/step - loss: 0.6266 - accuracy: 0.8039 - val_loss: 0.6130 - val_accuracy: 0.7824
Epoch 4/100
70/70 [==============================] - 6s 90ms/step - loss: 0.2283 - accuracy: 0.9325 - val_loss: 0.6124 - val_accuracy: 0.7950
Epoch 5/100
70/70 [==============================] - 6s 80ms/step - loss: 0.0734 - accuracy: 0.9798 - val_loss: 0.6385 - val_accuracy: 0.8147
Epoch 6/100
70/70 [==============================] - 5s 63ms/step - loss: 0.0401 - accuracy: 0.9924 - val_loss: 0.7465 - val_accuracy: 0.8022
Epoch 7/100
70/70 [==============================] - 4s 59ms/step - loss: 0.0165 - accuracy: 0.9973 - val_loss: 0.7934 - val_accuracy: 0.8112
E

In [ ]:
y_pred6 = model6.predict(x_test)
test_pr6 = y_pred6
test_te6 = y_test
test_pr6 = np.argmax(y_pred6, axis=1)
f1_mean = f1_score(test_te6, test_pr6, average=None).mean()
print(f1_mean)

29/29 [==============================] - 0s 7ms/step
0.802412036611363


# 제출 제작

In [ ]:
pred = model4.predict(X_test)

30/30 [==============================] - 0s 3ms/step


In [ ]:
test_df['text'][2]

"a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(input('정수 B의 값을 입력하시오.:'))\nc= int(input('정수 C의 값을 입력하시오.:'))\nd= int(input('정수 D의 값을 입력하시오.:'))\n\nmaximum = a\n\nif b&gt; maximum:\n    b= maximum\nif c&gt; maximum:\n    c= maximum\nif d&gt; maximum:\n    d= maximum\n    \nprint(f'최대값은 {maximum}입니다')\n\n왜 최초에 설정한 a값만 나오는지 알수 있을까요?"

In [ ]:
test_df['text'][631:641]

631    현재 22번 포트로 접근하는 SSH 접속이 거부되고 있습니다.\n현재 인스턴스의 보...
632    아까 제가 모르고 파일을 클릭하면서 mykey 메모장형식으로 바꿔버리게 되었는데 이...
633    마이키를 새로 받아도 텍스트 파일로 저장이 되어 실행이 안되는데 이럴때 다른 방법은...
634    안녕하세요\n이전 질문에서 ec2인스턴트 접속 후에 해야 한다는 답변을 받았었는데요...
635    0.0.0.0/0로 변경 후에도 안됩니당ㅜㅜㅜ 사용자 정의로  노트북 ip인 1.2...
636    라우팅 편집 버튼 클릭하여 NAT 게이트웨이 경로를 추가하려 하는데 아무것도 뜨지 ...
637    ssh명령어가 안돼서 실습진행을 못했습니다\n오후 실습을 위해 1대1문의 남깁니다....
638    튜터님들 고생이 많으십니다.\n\nAWS에서 어제 실습을 다 했던걸로 기억하는데 혹...
639                             캐시라는 개념에 대해서 아직 잘 모르겠습니다
640    교안 252쪽에 아키텍처를 보면 서버하나에 캐시도 하나씩 있는데 \n서버마다 캐시가...
Name: text, dtype: object

In [ ]:
test_df['text'][622]

'안녕하세요!\n실습 36페이지 복사한 명령어로 인스턴스 접속하는 부분 하는중인데\n사진과 같은 Connection timed out 이러한 에러로 접속이 안되는데\n어떤 문제 일까요?\n\n감사합니다!'

In [ ]:
test_cp = test_df.copy()
test_cp2 = test_df.copy()
test_cp

,id,text
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...
1,1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향..."
2,2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...
3,3,"def max4(a,b,c,d):\n maximum = a\n if b&..."
4,4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!
...,...,...
924,924,aice특강 ppt는 따로 안올려주시나요?
925,925,안녕하세요~!\n실습 중 궁금한 점이 있어 문의 드립니다. \n\n1. 라벨 인코더...
926,926,안녕하세요.\nkT AIDU에서 전에 Associate 시험에 대해 공부할 수 있는...
927,927,app 마다 urls.py 을 만들어서 관리하는게 더 좋은 코드인가요?\n아니면 s...


In [ ]:
test_cp.drop('text', axis=1, inplace=True)

In [ ]:
test_cp.head()

,id
0,0
1,1
2,2
3,3
4,4


In [ ]:
pred

array([[0.42191696, 0.00942025, 0.5592043 , 0.00707419, 0.00238433],
       [0.0387015 , 0.28399017, 0.06333221, 0.56617427, 0.0478019 ],
       [0.84491163, 0.12646015, 0.02351523, 0.0031939 , 0.00191905],
       ...,
       [0.76482177, 0.19372171, 0.02744256, 0.00950663, 0.00450735],
       [0.42419323, 0.20071028, 0.27474856, 0.07462937, 0.02571861],
       [0.5513269 , 0.11851777, 0.11114856, 0.19676206, 0.02224475]],
      dtype=float32)

In [ ]:
pred_A = np.argmax(pred, axis=1)

In [ ]:
pred_A

array([2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 3,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 4, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,

In [ ]:
test_cp['label'] = pred_A

In [ ]:
test_cp.head(30)

,id,label
0,0,2
1,1,3
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [ ]:
test_df['text'][19]

'금요일 수업의 경우 강의 영상은 월요일 17시에 업로드된다고 봤는데,\n혹시 일부만이라도 당일이나 익일 업로드는 불가할까요?\n주말동안 복습하고 싶어서요ㅜㅜ'

In [ ]:
test_cp.to_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/rmsdud5.csv', index= False)

In [ ]:
new_model = model.road

In [ ]:
# from tensorflow.keras.models import load_model
model4

In [ ]:
# # 모델 파일 로드
# model_path = '/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/f1_8640.h5'

# # 모델 로딩
# new_model = load_model(model_path)

# # 모델 사용
# answer_df = new_model.predict(X_test)

30/30 [==============================] - 0s 4ms/step


In [ ]:
# 0~21

In [ ]:
test_df['text'][19]

'금요일 수업의 경우 강의 영상은 월요일 17시에 업로드된다고 봤는데,\n혹시 일부만이라도 당일이나 익일 업로드는 불가할까요?\n주말동안 복습하고 싶어서요ㅜㅜ'

In [ ]:
test

AttributeError: ignored

In [ ]:
aa = test_df.copy()

In [ ]:
aa['new'] = new_answer

In [ ]:
aa.head(100)

,id,text,new
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...,3
1,1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향...",0
2,2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...,2
3,3,"def max4(a,b,c,d):\n maximum = a\n if b&...",4
4,4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!,4
...,...,...,...
95,95,"dict1 = {'x':[1,2,3,4,5,6,7,8,9,10,11,12], 'y1...",4
96,96,안녕하세요!! 에이블스쿨의 원활한 운영 지원에 감사드립니다!! \n\n다름아니라 이...,4
97,97,증가액 기준으로 상위 25개 행을 추출해 tmp 데이터프레임을 선언하는 문제 어떤 ...,4
98,98,figure size must be positive finite not [100.0...,4


In [ ]:
pd.set_option('display.max_columns', None) # 전체 열 보기
pd.set_option('display.max_rows', None) # 전체 행 보기

In [ ]:
test_cp2.shape

(929, 2)

# 다시 시작

In [ ]:
x_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/a/x_train_se (1).csv')
x_vala = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/a/x_val_se (1).csv')
y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/a/y_train.csv')
y_vala = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/a/y_val (1).csv')
x_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIVLE/ProjectAIVLE/project4/a/x_test_se (1).csv')

In [ ]:
x_train = np.array(x_train)
x_vala = np.array(x_vala)
y_train = np.array(y_train)
y_vala = np.array(y_vala)
x_test = np.array(x_test)

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=123)

In [ ]:
x_vala.shape, y_vala.shape

((736, 643), (742, 5))

In [ ]:
x_train.shape, y_train.shape

((2938, 643), (2964, 5))

## 모델링 1

In [ ]:
max_words = 20000 
embedding_dim = 128 ## 단어 embedding 차원
max_len = 643 ## 문장 최대 길이

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, restore_best_weights=True)

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add

# # 모델 정의
# inputs = Input(shape=(max_len,))
# embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)
# conv1 = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(embedding)
# dropout1 = Dropout(0.2)(conv1)
# # conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(dropout1)
# # residual = Add()([conv1, conv2])  # Residual connection
# pooling = GlobalMaxPooling1D()(dropout1)
# hl = Dense(32, activation='relu')(hl)
# hl = Dropout(0.2)(pooling)
# hl = keras.layers.Flatten()(hl)
# hl = Dense(32, activation='relu')(hl)

# outputs = Dense(5, activation='softmax')(hl)

# model4 = Model(inputs, outputs)

# # 모델 컴파일
# model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model4.summary()
# model4.fit(x_train, y_, epochs=10000, validation_split=0.2, verbose =1, callbacks=[es], batch_size=32)

In [ ]:
from sklearn.metrics import *
y_pred4 = model4.predict(x_vala)
test_pr4 = np.argmax(y_pred4, axis=1)
test_te4 = np.argmax(y_vala, axis=1)
f1_mean = f1_score(test_te4, test_pr4, average=None).mean()
print(f1_mean)
#0.8420327738533226

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add, Flatten, LSTM, Bidirectional

# 모델 정의
keras.backend.clear_session()
model6 = Sequential()
model6.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model6.add(LSTM(64))
# model6.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model6.add(Dropout(0.2))
model6.add(Dense(64, activation='relu'))
model6.add(Dropout(0.2))  # Dropout layer 추가
model6.add(keras.layers.Flatten())
model6.add(Dense(64, activation='relu'))
model6.add(Dense(5, activation='softmax'))

# 모델 컴파일
model6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model6.fit(x_train, y_train, epochs=10000, validation_split=0.2, verbose =1, callbacks=[es], batch_size=32)


In [ ]:
from sklearn.metrics import *
y_pred6 = model6.predict(x_vala)
test_pr6 = np.argmax(y_pred6, axis=1)
test_te6 = np.argmax(y_vala, axis=1)
f1_mean = f1_score(test_te6, test_pr6, average=None).mean()
print(f1_mean)
#0.8420327738533226

In [ ]:
f1_mean = f1_score(test_te4, test_pr4, average=None).mean()


In [ ]:
y_vala

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add, Flatten, LSTM, Bidirectional

# 입력 레이어 정의
input_layer = Input(shape=(max_len,), name='input_layer')

# 임베딩 레이어
embedding_layer = Embedding(input_dim=max_words, output_dim=128, input_length=max_len)(input_layer)

# Conv1D 레이어
conv1d_layer = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding_layer)

# Dropout 레이어
dropout = Dropout(0.2)(conv1d_layer)

# LSTM 레이어
lstm_layer = LSTM(units=64, activation='tanh')(dropout)

# Dense 레이어
dense_layer = Dense(units=16, activation='relu')(lstm_layer)

# 출력 레이어
output_layer = Dense(units=5, activation='softmax')(dense_layer)

# 모델 정의
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# 모델 정의
keras.backend.clear_session()
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

model.add(Conv1D(filters=64, kernel_size=5, activation='swish'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))  # Dropout layer 추가
model.add(keras.layers.Flatten())
model.add(Dense(units=64, activation='swish'))
# model5.add(Dense(units=64, activation='swish'))
# model5.add(Dropout(0.2))  # Dropout layer 추가
model.add(Dense(units=len(encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss=' categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, validation_split=0.1, verbose =1, callbacks=[es])

Epoch 1/100
83/83 [==============================] - 11s 107ms/step - loss: 1.4418 - accuracy: 0.4164 - val_loss: 1.3813 - val_accuracy: 0.4286
Epoch 2/100
83/83 [==============================] - 5s 62ms/step - loss: 1.3893 - accuracy: 0.4217 - val_loss: 1.3791 - val_accuracy: 0.4286
Epoch 3/100
83/83 [==============================] - 2s 30ms/step - loss: 1.3558 - accuracy: 0.4262 - val_loss: 1.3977 - val_accuracy: 0.4286
Epoch 4/100
83/83 [==============================] - 2s 26ms/step - loss: 1.2649 - accuracy: 0.4709 - val_loss: 1.4663 - val_accuracy: 0.4082
Epoch 5/100
83/83 [==============================] - 1s 16ms/step - loss: 1.0886 - accuracy: 0.5730 - val_loss: 1.5753 - val_accuracy: 0.3333
Epoch 6/100
83/83 [==============================] - 2s 20ms/step - loss: 0.8795 - accuracy: 0.6751 - val_loss: 1.7411 - val_accuracy: 0.2721
Epoch 7/100
83/83 [==============================] - 1s 9ms/step - loss: 0.6884 - accuracy: 0.7508 - val_loss: 2.0023 - val_accuracy: 0.3061
Epoch

In [ ]:
model.fit(x_train, y_train, epochs=10000, validation_split=0.2, verbose =1, callbacks=[es], batch_size=32)

In [ ]:
answer = model.predict(x_vala)
answer.shape

23/23 [==============================] - 0s 2ms/step


(736, 5)

In [ ]:
y_vala.shape

(742, 5)

In [ ]:
a = np.argmax(answer, axis=1)
b = np.argmax(y_vala, axis=1)

In [ ]:
a.shape, b.shape

((736,), (742,))

In [ ]:
f1_mean = f1_score(b, a, average=None).mean()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Add

# 모델 정의
inputs = Input(shape=(max_len,))
embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)
conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(embedding)
dropout1 = Dropout(0.2)(conv1)
# conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(dropout1)
# residual = Add()([conv1, conv2])  # Residual connection
pooling = GlobalMaxPooling1D()(dropout1)
hl = Dense(32, activation='relu')(hl)
hl = Dropout(0.2)(pooling)
hl = keras.layers.Flatten()(hl)
hl = Dense(32, activation='relu')(hl)

outputs = Dense(5, activation='softmax')(hl)

model4 = Model(inputs, outputs)

# 모델 컴파일
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

max_words = 20000 
embedding_dim = 128 ## 단어 embedding 차원
max_len = 643 ## 문장 최대 길이

# 모델 정의
keras.backend.clear_session()
model = Sequential()
model.add(Embedding( max_words, # 사용할 총 token개수
                    embedding_dim, # 한 token당 몇차원(몇개)의 특징값으로 바꾸어낼 것인가.
                    input_length=max_len )) # 통일한 문장 길이
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
a = 